In [1]:
import fiftyone as fo
from fiftyone import ViewField as F

# Load a small dataset
dataset = fo.zoo.load_zoo_dataset(
    "coco-2017",
    split="validation",
    label_types=["detections"],
    max_samples=10,
)

***Python 3.8 Deprecation Notice***
Python 3.8 will no longer be supported in new releases after October 1, 2024.
Please upgrade to Python 3.9 or later.
For additional details please see https://deprecation.voxel51.com


Found annotations at '/home/lradovan/fiftyone/coco-2017/raw/instances_val2017.json'
Sufficient images already downloaded
Existing download of split 'validation' is sufficient


***Python 3.8 Deprecation Notice***
Python 3.8 will no longer be supported in new releases after October 1, 2024.
Please upgrade to Python 3.9 or later.
For additional details please see https://deprecation.voxel51.com


Loading existing dataset 'coco-2017-validation-10'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


In [5]:
from libcom import FOPAHeatMapModel
from libcom.utils.process_image import make_image_grid, draw_bbox_on_image
import cv2
import os

fg_img = '/home/lradovan/workspace/libcom/tests/source/foreground/1.jpg'
fg_mask = '/home/lradovan/workspace/libcom/tests/source/foreground_mask/1.png'

#test_set = get_test_list_fopa_heatmap()

test_set = []
for sample in dataset[5:6]:
    print(sample.metadata.width)
    test_set.append({'foreground': fg_img, 'foreground_mask': fg_mask, 'background': sample.filepath})
    
result_dir = './results/'
net = FOPAHeatMapModel(device=0)

for pair in test_set[:1]:
    fg_img, fg_mask, bg_img = pair['foreground'], pair['foreground_mask'], pair['background']
    bboxes, heatmaps = net(fg_img, fg_mask, bg_img, cache_dir=os.path.join(result_dir, 'cache'), heatmap_dir=os.path.join(result_dir, 'heatmap'))
    img_name  = os.path.basename(bg_img).replace('.png', '.jpg')
    bbox = bboxes[0]
    bg_img  = draw_bbox_on_image(bg_img, [bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]])
    grid_img  = make_image_grid([bg_img, fg_img, heatmaps[0]])
    os.makedirs(os.path.join(result_dir, 'grid'), exist_ok=True)
    res_path  = os.path.join(result_dir, 'grid', img_name)
    cv2.imwrite(res_path, grid_img)
    print('save result to ', res_path)
print(f'end testing!')

640
save result to  ./results/grid/000000000785.jpg
end testing!


In [16]:
'''
Gets the largest bounding box from a background image
'''
def get_bounding_box(sample):
    detections = sample.ground_truth.detections
    if len(detections) > 0:
        print(f"\nImage path: {sample.filepath}")
        
        # Find largest bounding box
        largest_det = max(detections, key=lambda d: get_box_area(d.bounding_box))

        # get bbox
        bbox = largest_det.bounding_box
        width = sample.metadata.width
        height = sample.metadata.height
        
        x1 = int(bbox[0] * width)
        y1 = int(bbox[1] * height)
        x2 = int((bbox[0] + bbox[2]) * width)
        y2 = int((bbox[1] + bbox[3]) * height)

    return [x1, y1, x2, y2]

In [18]:
# takes a box of the form [x1, x2, y1, y2]
def get_box_area(box):
    return (box[1] - box[0]) * (box[3] - box[2])

In [14]:
# get overlap / background subject
def overlap_ratio(occluder_bb, occludee_bb):

    #top left and bottom right points
    occluder_x1, occluder_y1, occluder_x2, occluder_y2 = occluder_bb
    occludee_x1, occludee_y1, occludee_x2, occludee_y2 = occludee_bb

    # area of the foreground object
    occludee_area = (occludee_x2 - occludee_x1) * (occludee_y2 - occludee_y1)

    # area of the background object being covered by the foreground object
    overlap_area = max(0, min(occludee_x2, occluder_x2) - max(occludee_x1, occluder_x1)) * max(0, min(occludee_y2, occluder_y2) - max(occludee_y1, occluder_y1))

    # overlap over the total background object area
    return overlap_area / occludee_area

In [53]:
# we need some way to get the bounding boxes for the occluders....
import random

def get_bbox_list(bg_bbox, fg_w, fg_h):

    bboxes = []

    occludee_x1 = bg_bbox[0]
    occludee_y1 = bg_bbox[1]
    occludee_x2 = bg_bbox[2]
    occludee_y2 = bg_bbox[3]

    num_boxes = 5 # iterate over multiple boxes
    overlap_threshold = .2 # ensure a minimum amount of overlap

    # print(bg_bbox, fg_w, fg_h)
    
    for _ in range(num_boxes):
        # top left corner of the occluder bounding box:

        occluder_x1 = random.randint(max(0, occludee_x1 - fg_w), occludee_x2) # overlapping in the x-direction
        occluder_y1 = random.randint(max(0, occludee_y1 - fg_h), occludee_y2) # overlapping in the y-direction

        # TODO: Fix so doesnt exceed background image
        occluder_x2 = occluder_x1 + fg_w
        occluder_y2 = occluder_y1 + fg_h

        occluder_bb = [occluder_x1, occluder_y1, occluder_x2, occluder_y2]
        occluded_ratio = overlap_ratio(occluder_bb, bg_bbox)

        if occluded_ratio >= overlap_threshold:
            bboxes.append([occluder_x1, occluder_y1, occluder_x2, occluder_y2])
    
    return bboxes

In [55]:
# IN PROGRESS
# TODO: Fix broadcast issue between bounding box and mask (they are different sizes!!!)

from libcom.utils.process_image import *
from libcom.utils.environment import *
from libcom import OPAScoreModel
from libcom import get_composite_image
from libcom.utils.process_image import make_image_grid
import cv2
import csv
from PIL import Image

net = OPAScoreModel(device=0, model_type='SimOPA')

# for each scale, take the highest OPA score
fg_scale_num = 16

# trying with just one foreground image (teddy bear) and one background image (skier)
foreground_image = '/home/lradovan/workspace/libcom/tests/source/foreground/1.jpg'
foreground_mask = '/home/lradovan/workspace/libcom/tests/source/foreground_mask/1.png'

bg_imgs = []
for sample in dataset[5:6]:
    bg_imgs.append([sample.filepath, get_bounding_box(sample)])

background_image = bg_imgs[0][0]

cache_dir = './cache'

from libcom.fopa_heat_map.source.prepare_multi_fg_scales import prepare_multi_fg_scales
scaled_fg_dir, scaled_mask_dir, csv_path = prepare_multi_fg_scales(cache_dir, foreground_image, foreground_mask, background_image, fg_scale_num)
 
# iterate over the different foreground scales
with open(csv_path, mode='r', newline='') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    score = 0
    res = [-1, -1]
    for row in csv_reader:
        fg_name   = '{}_{}_{}_{}.jpg'.format(row["fg_name"].split(".")[0],row["bg_name"].split(".")[0],int(row["newWidth"]),int(row["newHeight"]))
        mask_name = '{}_{}_{}_{}.jpg'.format(row["fg_name"].split(".")[0],row["bg_name"].split(".")[0],int(row["newWidth"]),int(row["newHeight"]))
        scale     = row['scale']
        fg_w = int(row['newWidth'])
        fg_h = int(row['newHeight'])
        
        save_name = fg_name.split(".")[0] + '_' + str(scale) + '.jpg'

        bg_img    = read_image_pil(background_image)
        fg_img    = read_image_pil(os.path.join(scaled_fg_dir, fg_name))
        fg_mask   = read_mask_pil(os.path.join(scaled_mask_dir, mask_name))
        bbox_list = get_bbox_list(bg_imgs[0][1], fg_w, fg_h)

        print(fg_w, fg_h)
        print(bbox_list)

        for bbox in bbox_list:
            comp, comp_mask = get_composite_image(fg_img, fg_mask, bg_img, bbox)
            bbox_score = net(comp, comp_mask)
            if bbox_score > score:
                res = comp, comp_mask
                score = bbox_score

    print(score)
        # grid_img  = make_image_grid(res)
        # cv2.imwrite('/home/lradovan/results/', grid_img)


Image path: /home/lradovan/fiftyone/coco-2017/validation/data/000000000785.jpg
23 23
[]
49 47
[]
73 70
[]
98 94
[]
123 118
[]
147 141
[[361, 51, 508, 192], [296, 245, 443, 386], [322, 67, 469, 208]]
172 165
[[397, 74, 569, 239], [283, 262, 455, 427], [209, 183, 381, 348]]


ValueError: could not broadcast input array from shape (165,172) into shape (163,172)